In [ ]:
# !pip install -e /Users/esummer/workspace/python/my-python-utils-main

In [1]:
!pip install slugify

!pip install pandas
!pip install selenium
!pip install openpyxl

!pip install requests pandas xmltodict
!pip install etree
!pip install lxml

!pip install --upgrade pip


ERROR: Could not find a version that satisfies the requirement etree (from versions: none)
ERROR: No matching distribution found for etree


In [2]:
import json
import os

from slugify import slugify
import csv

import logging
import time
import random
import pandas as pd
from datetime import datetime
from zoneinfo import ZoneInfo

import requests
from selenium import webdriver
from selenium.webdriver.common.by import By

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.firefox_profile import FirefoxProfile

import pandas as pd
import time
import requests
import xml.etree.ElementTree as ET

from zoneinfo import ZoneInfo

import re

## 국가유산청 Open API

In [ ]:
import pandas as pd
import time
import requests
import xml.etree.ElementTree as ET

df = pd.DataFrame()
error_urls = []
for i in range(1, int(17737 / 50) +1 ):
    url = f'http://www.khs.go.kr/cha/SearchKindOpenapiList.do?pageUnit=50&pageIndex={i}'

    try:
        # 2. 데이터 가져오기 (requests 이용)
        response = requests.get(url)
        response.raise_for_status()  # 200 OK가 아니면 에러 발생

        # 3. XML 데이터를 데이터프레임으로 변환
        # 국가유산청 API 결과에서 각 항목은 <item> 태그 안에 들어있으므로 xpath를 지정합니다.
        df_a = pd.read_xml(response.content, xpath=".//item")
        df = pd.concat([df, df_a], ignore_index=True)

        df_a.to_excel(f'{i:04d}_SearchKindOpenapiList.xlsx')
        # 4. 결과 확인
        print(f"데이터 크기: {df.shape}")
        # print(df.head())
        time.sleep(1)
    except Exception as e:
        print(f"오류 발생: {e}")
        error_urls.append(url)
    pass


In [ ]:
df.to_excel("국가유산.xlsx", index=False)

## 국가유산 검색 상세

In [ ]:
df_base = pd.read_excel("국가유산.xlsx")

In [ ]:
# import ET
from pathlib import Path
from datetime import datetime
import pytz

# 한국 타임존 설정
korea_timezone = pytz.timezone('Asia/Seoul')

step_count = 3000
step = 1 +1+1+1+1+1

start = ((step -1) * step_count)

print(start, start + step_count)
for index, row in df_base.iterrows() :

    if index < start + step_count and index >= start:
        ccbaKdcd = row['ccbaKdcd']
        ccbaAsno = row['ccbaAsno']
        ccbaCtcd = row['ccbaCtcd']
        file_name = row['ccbaMnm1']

        ccbaAsno = f'0000000{ccbaAsno}'
        ccbaAsno = ccbaAsno[-13:]

        surl = f"https://www.khs.go.kr/cha/SearchKindOpenapiDt.do?ccbaKdcd={ccbaKdcd}&ccbaAsno={ccbaAsno}&ccbaCtcd={ccbaCtcd}"

        if (index % 50 == 0):
            # 현재 시간 가져오기
            now_korea = datetime.now(korea_timezone)
            # 출력 형식 지정
            formatted_time = now_korea.strftime('%Y-%m-%d %H:%M:%S')
            print("", formatted_time, index+1, surl)

        try:
            file_path = Path(f'./detail/{index:05d}_상세_{file_name}.xlsx')

            # 1. 파일 또는 디렉토리 존재 여부 전체
            if file_path.exists():
                continue

            print("*", end="")

            # 2. 데이터 가져오기 (requests 이용)
            response = requests.get(surl)
            response.raise_for_status()

            df_a = pd.read_xml(response.content, xpath=".//item")
            df = pd.concat([df, df_a], ignore_index=True)

            df_a.to_excel(file_path)

        except Exception as ex:
            print(ex)
            print(index, surl)

## 국가유산 목록에서 이미지파일 조회

In [ ]:
df = pd.read_excel("국가유산.xlsx")
# df.head()

In [ ]:
# import ET
from pathlib import Path
step = 1000
step = 10

start = 1000 +500 +500 +500 +500 +500 +1000 +1000 +step+step
start = 11500+step
start = 6500
start = 17500
start = 0

for index, row in df.iterrows() :

    if index < start + step and index >= start:
        ccbaKdcd = row['ccbaKdcd']
        ccbaAsno = row['ccbaAsno']
        ccbaCtcd = row['ccbaCtcd']
        file_name = row['ccbaMnm1']

        ccbaAsno = f'0000000{ccbaAsno}'
        ccbaAsno = ccbaAsno[-13:]

        surl = f'https://www.khs.go.kr/cha/SearchImageOpenapi.do?ccbaKdcd={ccbaKdcd}&ccbaAsno={ccbaAsno}&ccbaCtcd={ccbaCtcd}'
        if(index % 20 == 0):
            # print(index, surl)
            pass
        try:
            file_path = Path(f'./output/{index:05d}_image_{file_name}.xlsx')

            # 1. 파일 또는 디렉토리 존재 여부 전체
            if file_path.exists():
                continue


            # 2. 데이터 가져오기 (requests 이용)
            response = requests.get(surl)
            response.raise_for_status()

            items = response.content

            root = ET.fromstring(response.content)

            result_list = []
            current_item = {}

            # 2. 모든 하위 요소를 순회하며 sn을 기준으로 그룹화
            for child in root:

                # print(child.tag, child.text)

                if child.tag == 'item':
                    for child2 in child:
                        # print(child2.tag, child2.text)
                        # 새로운 <sn>을 만나면 이전까지 쌓인 current_item을 리스트에 추가
                        if child2.tag == 'sn' and current_item:
                            result_list.append(current_item)
                            current_item = {}  # 초기화

                        # 데이터 추가 (태그 이름을 키로, 텍스트를 값으로)
                        current_item[child2.tag] = child2.text

                    # 3. 마지막에 남은 항목 추가
                    if current_item:
                        result_list.append(current_item)

            if(index % 20 == 0):
                print(result_list)

            df2 = pd.DataFrame(result_list)
            df2.to_excel(f'./output/{index:05d}_image_{file_name}.xlsx')
        except Exception as ex:
            print(ex)
            print(index, surl)


In [ ]:
df = pd.read_excel('이미지_국가유산___.xlsx')

In [ ]:
words = "석탑,입상,좌상,칠성도,석등,극락전,극락보전,전탑,무량수전,불국사,해인사,송광사,통도사,백담사,동화사,벽화,대웅전,승탑,법주사,여래,국사탑,사탑,괘불탱,탱화,동종,반가상,미륵불,보살,나한상,괘불".split(",")
words = "교회,성당,예배".split(",")
words = "나무,수림".split(",")
words = "박물관".split(",")

files = []

for index, row in df.iterrows() :
    ccbaKdcd = row['ccbaKdcd']
    ccbaAsno = row['ccbaAsno']
    ccbaCtcd = row['ccbaCtcd']

    # df.at[index, 'images'] = 0

    file_name = row['ccbaMnm1']
    if any(word in file_name for word in words):
        readfile = f'{index:05d}_image_{file_name}.xlsx'

        directory = file_name.replace(":","^").replace("/","&").replace(" ","_").replace("\t","_")

        images = []

        try:
            df2 = pd.read_excel(f"./output/{readfile}")
            for ii, rr in df2.iterrows():
                # if ii > 9:
                #     break
                data = {}
                data['url'] = rr['imageUrl']

                if(data['url'].find('no_image') > -1):
                    continue

                ext = data['url'].split('.')[-1]
                sn = rr['sn']
                name = f"{rr['ccimDesc']}".replace(":","^").replace("/","&").replace(" ","_").replace("\t","_")
                data['name'] = f'{directory}@@{name}##{sn}.{ext}'
                images.append(data)
        except Exception as ex:
            print(ex)
        finally:
            if len(images) > 0:
                df.at[index, 'read_file'] = readfile
                df.at[index, 'images'] = len(images)
                df.at[index, f'images_type'] = "|".join(words[:3])
                files.append(readfile)

    # if(index > 10000):
    #     break

df3 = pd.DataFrame(files, columns=['Name'])
df3.to_excel(f'{"".join(words)}___.xlsx')
df.to_excel(f'이미지_국가유산___.xlsx')
print("파일갯수",len(files),"\n", "Latest\n","\n".join(files[-10:]))

In [ ]:
# 1. 필터링할 단어 리스트 정의
words = "석탑,입상,좌상,칠성도,석등,극락전,극락보전,전탑,무량수전,불국사,해인사,송광사,통도사,백담사,동화사,벽화,대웅전,승탑,법주사,여래,국사탑,사탑,괘불탱,탱화,동종,반가상,미륵불,보살,나한상,괘불".split(",")

# 2. 정규표현식 패턴 생성 (단어들 중 하나라도 포함된 경우)
pattern = "|".join(words)

# 3. ccbaMnm1 컬럼에서 해당 단어가 포함된 데이터 필터링
# na=False를 설정하여 NaN 값이 있는 경우 에러 방지
filtered_df = df[df['ccbaMnm1'].str.contains(pattern, na=False)].copy()

result_df = filtered_df.groupby(['ccbaCtcdNm', 'ccsiName', 'ccbaAdmin'])['ccbaMnm1'].agg(
    count='count',
    all_name=lambda x: "|".join(x.astype(str))
).reset_index()
result_df.to_excel('지역_국가유산___.xlsx')

#
# df_area = df.groupby(['ccbaCtcdNm', 'ccsiName', 'ccbaAdmin']).agg(
#     count_ccbaMnm1=('ccbaMnm1', 'count'),  # 개수 검사
#     all_name=('ccbaMnm1', lambda x: "|".join(x.astype(str)))  # "|" 구분자로 합치기
# ).reset_index()
# df_area.to_excel('지역_국가유산___.xlsx')